In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os
from pathlib import Path
from scipy import stats
from statannot import add_stat_annotation

font = {'family' : 'Helvetica',
        'weight' : 'normal',
        'size'   : 8}
matplotlib.rc('font', **font)

In [ ]:
drug_folders = ['Quin-10uM', 'Flec-3uM', 'Lido-100uM', 'control-combined']
#drug_folders = ['Quin-10uM', 'Lido-100uM', 'control-combined']
df_list = []
for drug in drug_folders:
    df = pd.DataFrame()
    for root, dirs, files in os.walk(Path('./'+drug)):
        if len(files)>0:
            if files[0].endswith('.txt'):
                folder = (root)
                for file in files:
                    file_df = pd.read_csv(Path(root+'/'+file))
                    # some silly string manipulation to get the time point
                    timepoint = file[0:-4]
                    file_df = file_df.dropna(axis=0, how='any')
                    file_df['timepoint'] = [timepoint]*file_df.shape[0]
                    file_df['folder'] = [folder]*file_df.shape[0]
                    df = pd.concat([df, file_df], ignore_index = True)
    df = df[df['SP_Amp_avg'] > 0.3]
    df = df[df['PP_Amp_avg'] > 0.5]
    df = df[df['t_rise_avg'] < 0.2]
    df = df[df['APD10_avg'] > 0.005]
    #only keep paired data
    df = df.groupby(['labels', 'folder']).filter(lambda x: len(x)>1)
    # compute triangulation (ratio method) and overpotential
    df['tri_ratio'] = (df['APD90_avg']-df['APD50_avg'])/(df['APD50_avg']-df['APD10_avg'])
    df['overpotential'] = (df['PP_Amp_avg']-df['SP_Amp_avg'])/df['SP_Amp_avg']
    df['start_min'] = (df['PP_Amp_avg']-df['SP_Amp_avg'])
    df['atrial'] = ((df['APD50_avg']/df['APD90_avg']) < 0.5)
    df['period'] = 1/df['frequency']
    df['c_APD90_Bazett'] = df['APD90_avg']/(np.sqrt(df['period']))
    df['c_APD90_Fridericia'] = df['APD90_avg']/(np.cbrt(df['period']))
#    df['c_APD50'] = df['APD50_avg']/(np.sqrt(df['period']))
#    df['c_APD10'] = df['APD10_avg']/(np.sqrt(df['period']))
    df['RL_CL'] = df['max_min_time']/df['period']
    df['c_RL_Bazett'] = df['max_min_time']/(np.sqrt(df['period']))
    df['c_RL_Fridericia'] = df['max_min_time']/(np.cbrt(df['period']))
    df_list.append(df) 

#datas = ['APD90_avg', 'APD50_avg', 'APD10_avg', 't_rise_avg', 'frequency', 'overpotential','triangle', 'RL_CL', 'period', 'max_min_time']
#labels = ['APD90', 'APD50', 'APD10', 't-rise', 'frequency', 'overpotential','triangulation', 'RL-to-CL', 'CL', 'RL']
#units = ['/s','/s','/s','/s','', '','/s', '', '/s', '/s']
datas = ['APD90_avg', 'c_APD90_Bazett', 'c_APD90_Fridericia', 'RL_CL', 'c_RL_Bazett', 'c_RL_Fridericia']
labels = ['APD90', 'cAPD90_Bazett', 'cAPD90_Fridericia', 'RL-to-CL', 'cRL_Bazett', 'cRL_Fridericia']
units = ['/s', '', '', '', '', '', '/s']
order = ['before', 'after']
box_pairs = [('before', 'after')]

In [ ]:
annotate = True
#plot stripplot with pointplot overlay
for j in range(len(df_list)):
    df = df_list[j]
    drug = drug_folders[j]
    for i in range(len(datas)):
        y_data = datas[i]
        ylabel = labels[i]
        unit = units[i]
        fig, ax = plt.subplots(figsize=(1.5,3), dpi=300)
        if ylabel != 'frequency':
            sns.stripplot(data = df, x = 'timepoint', y = y_data, order=order, size=3, color='black', zorder=1, ax=ax)
        sns.pointplot(data = df, x = 'timepoint', y = y_data , order=order, ci='sd', scale=0.5, errwidth=1.5, capsize=0.1, zorder=2, ax=ax)      
        sns.despine()
        ax.set_xlabel('Timepoint')
        ax.set_ylabel(ylabel+unit)
        #ax.set_title('Average '+ ylabel +' over Time')
        savepath = drug+'/'+drug+'_'+ylabel
        if annotate:
            test = add_stat_annotation(ax, data=df, x='timepoint', y=y_data, order=order,
                                       box_pairs=box_pairs, test='t-test_paired', text_format='star',
                                       line_height=0.02, text_offset=0.01, line_offset=0.01, line_offset_to_box=0.02,
                                       loc='inside', linewidth=1, verbose=0)
        fig.savefig(Path(savepath+'.png'), bbox_inches='tight')
        plt.close()

In [ ]:
sort_by_folder = False
#plot individual datapoints with connections
for j in range(len(df_list)):
    df = df_list[j]
    drug = drug_folders[j]
    for i in range(len(datas)):
        y_data = datas[i]
        ylabel = labels[i]
        unit = units[i]
        fig, ax = plt.subplots(figsize=(1.5,3), dpi=300)
        savepath = drug+'/'+drug+'_'+ylabel+'_scatter'
        if sort_by_folder:
            savepath = savepath+'_byfolder'
            sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, hue='folder', ax=ax)
            ax.get_legend().remove()
        else:
            sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, color='black', ax=ax)
        sns.despine()
        #plot connections
        idx0 = 0
        idx1 = 1
        locs1 = ax.get_children()[idx0].get_offsets()
        locs2 = ax.get_children()[idx1].get_offsets()
        for i in range(locs1.shape[0]):
            x = [locs1[i, 0], locs2[i, 0]]
            y = [locs1[i, 1], locs2[i, 1]]
            ax.plot(x, y, color="black", linewidth=0.5, alpha=0.1)
        ax.set_xlabel('Timepoint')
        ax.set_ylabel(ylabel+unit)
        #ax.set_title('Average '+ ylabel +' over Time')        
        test = add_stat_annotation(ax, data=df, x='timepoint', y=y_data, order=order,
                                   box_pairs=box_pairs, test='t-test_ind', text_format='star',
                                   line_height=0.02, text_offset=0.01, line_offset=0.01, line_offset_to_box=0.02,
                                   loc='inside', linewidth=1, verbose=0)
        fig.savefig(Path(savepath+'.png'), bbox_inches='tight')
        plt.close()

In [ ]:
sort_by_folder = True
#plot individual datapoints with connections by folder
for j in range(len(df_list)):
    df = df_list[j]
    drug = drug_folders[j]
    for i in range(len(datas)):
        y_data = datas[i]
        ylabel = labels[i]
        unit = units[i]
        fig, ax = plt.subplots(figsize=(1.5,3), dpi=300)
        savepath = drug+'/'+drug+'_'+ylabel+'_scatter'
        if sort_by_folder:
            savepath = savepath+'_byfolder'
            sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, hue='folder', ax=ax)
            ax.get_legend().remove()
        else:
            sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, color='black', ax=ax)
        sns.despine()
        #plot connections
        idx0 = 0
        idx1 = 1
        locs1 = ax.get_children()[idx0].get_offsets()
        locs2 = ax.get_children()[idx1].get_offsets()
        for i in range(locs1.shape[0]):
            x = [locs1[i, 0], locs2[i, 0]]
            y = [locs1[i, 1], locs2[i, 1]]
            ax.plot(x, y, color="black", linewidth=0.5, alpha=0.1)
        ax.set_xlabel('Timepoint')
        ax.set_ylabel(ylabel+unit)
        #ax.set_title('Average '+ ylabel +' over Time')        
        test = add_stat_annotation(ax, data=df, x='timepoint', y=y_data, order=order,
                                   box_pairs=box_pairs, test='t-test_ind', text_format='star',
                                   line_height=0.02, text_offset=0.01, line_offset=0.01, line_offset_to_box=0.02,
                                   loc='inside', linewidth=1, verbose=0)
        
        fig.savefig(Path(savepath+'.png'), bbox_inches='tight')
        plt.close()

In [ ]:
#plot with ylim
sort_by_folder = False
pointplot_overlay=False
violinplot_overlay=True
for j in range(len(df_list)):
    df = df_list[j]
    drug = drug_folders[j]
    datas = ['APD90_avg','RL_CL', 'period', 'c_APD90_Bazett', 't_rise_avg']
    labels = ['APD90', 'RL-to-CL', 'CL', 'cAPD90', 't-depol']
    units = ['/s', '' , '/s' ,'', '/s']
    order = ['before', 'after']
    box_pairs = [('before', 'after')]
    ylim = [(0, 1.4),(0, 0.6), (0, 7),(0.2, 0.75),(0, 0.05)]
    print(drug_folders[j])
    print(str(df.shape[0]//2)+ ' datapoints')
    for i in range(len(datas)):
        y_data = datas[i]
        print(y_data)
        ylabel = labels[i]
        unit = units[i]
        ylims = ylim[i]
        fig, ax = plt.subplots(figsize=(1,2), dpi=600)
        savepath = drug+'/'+drug+'_'+ylabel+'_scaled'

        if pointplot_overlay:
            savepath=savepath+'_withsd'
            sns.pointplot(data = df, x = 'timepoint', y = y_data , order=order, ci='sd', scale=0.3, errwidth=1, capsize=0.05, zorder=2, ax=ax)
            if sort_by_folder:
                savepath = savepath+'_byfolder'
                sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, hue='folder', ax=ax, zorder=1)
                ax.get_legend().remove()
            else:
                sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, color='black', ax=ax, zorder=1)
            #plot connections
            idx0 = 1
            idx1 = 2
            locs1 = ax.get_children()[idx0].get_offsets()
            locs2 = ax.get_children()[idx1].get_offsets()
            for i in range(locs1.shape[0]):
                x = [locs1[i, 0], locs2[i, 0]]
                y = [locs1[i, 1], locs2[i, 1]]
                ax.plot(x, y, color="black", linewidth=0.5, alpha=0.1)
        elif violinplot_overlay:
            savepath=savepath+'_violin'
            sns.violinplot(data = df, x = 'timepoint', y = y_data , order=order, zorder=2, linewidth=0.5, saturation=0.5, color='steelblue', ax=ax)
            if sort_by_folder:
                savepath = savepath+'_byfolder'
                sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, hue='folder', ax=ax, zorder=1)
                ax.get_legend().remove()
            else:
                sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, color='black', ax=ax, zorder=1)
            #plot connections
            idx0 = 4
            idx1 = 5
            locs1 = ax.get_children()[idx0].get_offsets()
            locs2 = ax.get_children()[idx1].get_offsets()
            for i in range(locs1.shape[0]):
                x = [locs1[i, 0], locs2[i, 0]]
                y = [locs1[i, 1], locs2[i, 1]]
                ax.plot(x, y, color="black", linewidth=0.5, alpha=0.1)
        else:
            savepath=savepath+'_scatter'
            if sort_by_folder:
                savepath = savepath+'_byfolder'
                sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, hue='folder', ax=ax, zorder=1)
                #ax.get_legend.remove()
                plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
            else:
                sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, color='black', ax=ax, zorder=1)    

            #plot connections
            idx0 = 0
            idx1 = 1
            locs1 = ax.get_children()[idx0].get_offsets()
            locs2 = ax.get_children()[idx1].get_offsets()
            for i in range(locs1.shape[0]):
                x = [locs1[i, 0], locs2[i, 0]]
                y = [locs1[i, 1], locs2[i, 1]]
                ax.plot(x, y, color="black", linewidth=0.5, alpha=0.1)

        sns.despine()
        ax.set_xlabel('Timepoint')
        ax.set_ylabel(ylabel+unit)
        ax.set_ylim(ylims[0], ylims[1])
        
        bef = df[df['timepoint'] == 'before']
        aft = df[df['timepoint'] == 'after']
        diff = aft[y_data].to_numpy() - bef[y_data].to_numpy()
        test_type = 't-test_paired'
        if stats.shapiro(diff)[1] < 0.05:
            test_type = 'Wilcoxon'
        test = add_stat_annotation(ax, data=df, x='timepoint', y=y_data, order=order,
                                   box_pairs=box_pairs, test=test_type, text_format='star',
                                   line_height=0.02, text_offset=0.01, line_offset=0.01, line_offset_to_box=0.02,
                                   loc='outside', linewidth=1, verbose=0)
        
    
#        print(stats.shapiro(diff))
#        print(stats.skewtest(diff))
#        print(stats.kurtosistest(diff))
#        print(stats.wilcoxon(bef[y_data].to_numpy(), aft[y_data].to_numpy()))
#        print(stats.ttest_rel(bef[y_data].to_numpy(), aft[y_data].to_numpy()))

#        fig.savefig(Path(savepath+'.png'), bbox_inches='tight')
        plt.savefig(Path(savepath+'.svg'), format='svg', bbox_inches='tight')
        plt.close()

In [ ]:
# plot t-rise filtering <0.015
sort_by_folder = False
pointplot_overlay = False
violinplot_overlay = True

for i in range(len(df_list)):
    df1 = df_list[i]
    drug = drug_folders[i]
    df2 = df1[df1['timepoint']=='before']
    df2 = df2[df2['t_rise_avg'] > 0.015]
    df = pd.merge(df1, df2, on=['labels','folder', 't_rise_avg', 'timepoint'], how='outer', indicator=True)
    df = df.query("_merge != 'both'").drop('_merge', axis=1)
    df = df.groupby(['labels', 'folder']).filter(lambda x: len(x)>1).reset_index(drop=True)

    print('dropped '+str(df2.shape[0])+' datapoints')
    print('remaining: '+str(df.shape[0]//2)+ ' datapoints')

    order = ['before', 'after']
    box_pairs = [('before', 'after')]
    y_data = 't_rise_avg'
    ylabel = 'depolarization time'
    unit = '/s'
    ylims = (0, 0.05)
    fig, ax = plt.subplots(figsize=(1,2), dpi=600)
    savepath = drug+'/'+drug+'_'+ylabel+'_scaled_lessthan0.015'

    if pointplot_overlay:
        savepath=savepath+'_withsd'
        sns.pointplot(data = df, x = 'timepoint', y = y_data , order=order, ci='sd', scale=0.3, errwidth=1, capsize=0.05, zorder=2, ax=ax)
        if sort_by_folder:
            savepath = savepath+'_byfolder'
            sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, hue='folder', ax=ax, zorder=1)
            ax.get_legend().remove()
        else:
            sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, color='black', ax=ax, zorder=1)
        #plot connections
        idx0 = 1
        idx1 = 2
        locs1 = ax.get_children()[idx0].get_offsets()
        locs2 = ax.get_children()[idx1].get_offsets()
        for i in range(locs1.shape[0]):
            x = [locs1[i, 0], locs2[i, 0]]
            y = [locs1[i, 1], locs2[i, 1]]
            ax.plot(x, y, color="black", linewidth=0.5, alpha=0.1)
    elif violinplot_overlay:
        savepath=savepath+'_violin'
        sns.violinplot(data = df, x = 'timepoint', y = y_data , order=order, zorder=2, linewidth=0.5, saturation=0.2, color='steelblue', ax=ax)
        if sort_by_folder:
            savepath = savepath+'_byfolder'
            sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, hue='folder', ax=ax, zorder=1)
            ax.get_legend().remove()
        else:
            sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, color='black', ax=ax, zorder=1)
        #plot connections
        idx0 = 4
        idx1 = 5
        locs1 = ax.get_children()[idx0].get_offsets()
        locs2 = ax.get_children()[idx1].get_offsets()
        for i in range(locs1.shape[0]):
            x = [locs1[i, 0], locs2[i, 0]]
            y = [locs1[i, 1], locs2[i, 1]]
            ax.plot(x, y, color="black", linewidth=0.5, alpha=0.1)
    else:
        savepath=savepath+'_scatter'
        if sort_by_folder:
            savepath = savepath+'_byfolder'
            sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, hue='folder', ax=ax, zorder=1)
            #ax.get_legend().remove()
            plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
        else:
            sns.stripplot(data = df, x = 'timepoint', y=y_data, order=order, size=3, color='black', ax=ax, zorder=1)    

        #plot connections
        idx0 = 0
        idx1 = 1
        locs1 = ax.get_children()[idx0].get_offsets()
        locs2 = ax.get_children()[idx1].get_offsets()
        for i in range(locs1.shape[0]):
            x = [locs1[i, 0], locs2[i, 0]]
            y = [locs1[i, 1], locs2[i, 1]]
            ax.plot(x, y, color="black", linewidth=0.5, alpha=0.1)

    sns.despine()
    ax.set_xlabel('Timepoint')
    ax.set_ylabel(ylabel+unit)
    ax.set_ylim(ylims[0], ylims[1])
    
    bef = df[df['timepoint'] == 'before']
    aft = df[df['timepoint'] == 'after']
    diff = aft[y_data].to_numpy() - bef[y_data].to_numpy()
    test_type = 't-test_paired'
    if stats.shapiro(diff)[1] < 0.05:
        test_type = 'Wilcoxon'
    test = add_stat_annotation(ax, data=df, x='timepoint', y=y_data, order=order,
                               box_pairs=box_pairs, test=test_type, text_format='star',
                               line_height=0.02, text_offset=0.01, line_offset=0.01, line_offset_to_box=0.02,
                               loc='outside', linewidth=1, verbose=0)

    #print(stats.shapiro(diff))
    #print(stats.skewtest(diff))
    #print(stats.kurtosistest(diff))
    #print(stats.wilcoxon(bef[y_data].to_numpy(), aft[y_data].to_numpy()))
    #print(stats.ttest_rel(bef[y_data].to_numpy(), aft[y_data].to_numpy()))
    #fig.savefig(Path(savepath+'.png'), bbox_inches='tight')
    plt.savefig(Path(savepath+'.svg'), format='svg', bbox_inches='tight')
    
    plt.close()


In [ ]:
#plot diffs or ratios
drop_t = False
pointplot_overlay=False
violinplot_overlay=True
plot_type = 'diff'
datas = ['APD90_avg','RL_CL', 'period', 'c_APD90_Bazett', 't_rise_avg']
labels = ['APD90', 'RL-to-CL', 'CL', 'cAPD90', 't-depol']
units = ['/s', '' , '/s' ,'', '/s']
box_pairs = [('Quin', 'Control'),
            ('Flec', 'Control'),
            ('Lido', 'Control')]
dlabels = ['Quin', 'Flec', 'Lido', 'Control']
dfs = df_list
if drop_t:
    dfs = []
    for i in range(len(df_list)):
        df1 = df_list[i]
        drug = drug_folders[i]
        df2 = df1[df1['timepoint']=='before']
        df2 = df2[df2['t_rise_avg'] > 0.015]
        df = pd.merge(df1, df2, on=['labels','folder', 'APD90_avg','RL_CL', 'period', 
                                    'c_APD90_Bazett','t_rise_avg', 'timepoint'], how='outer', indicator=True)
        df = df.query("_merge != 'both'").drop('_merge', axis=1)
        df = df.groupby(['labels', 'folder']).filter(lambda x: len(x)>1).reset_index(drop=True)

        print('dropped '+str(df2.shape[0])+' datapoints')
        print('remaining: '+str(df.shape[0]//2)+ ' datapoints')
        dfs.append(df)
for i in range(len(datas)):
    y_data = datas[i]
    ylabel = labels[i]
    unit = units[i]
    df = pd.DataFrame()
    for j in range(len(dfs)):
        df_temp = dfs[j]
        drug = dlabels[j]        
        bef = df_temp[df_temp['timepoint'] == 'before']
        aft = df_temp[df_temp['timepoint'] == 'after']
        
        # compute diff, ratio for data
        diff = aft[y_data].to_numpy() - bef[y_data].to_numpy()
        ratio = aft[y_data].to_numpy()/bef[y_data].to_numpy()

        print(ylabel + ' ' + drug + ' diff')
        print(stats.bootstrap([diff], np.mean, method="percentile"))
        
        #print(ylabel + ' ' + drug + ' ratio')
        #print(stats.bootstrap([ratio], np.mean, method="percentile"))
        #print(stats.shapiro(ratio))
        #print(stats.kstest(ratio, 'norm'))
        #print(stats.skewtest(ratio))
        #print(stats.kurtosistest(ratio))
        
        df_drug = pd.DataFrame()
        df_drug['diff'] = diff
        df_drug['ratio'] = ratio
        df_drug['drug'] = [drug]*len(diff)
        df = pd.concat([df, df_drug], ignore_index=True)
    fig, ax = plt.subplots(figsize=(4,2), dpi=600)
    savepath = ylabel+'_'+plot_type+'_comparison'
    if drop_t:
        savepath = savepath+"_trimmed"
    if pointplot_overlay:
        savepath=savepath+'_withsd'
        sns.pointplot(data = df, x = 'drug', y = plot_type , order=dlabels, ci='sd', scale=0.3, errwidth=1, capsize=0.05, zorder=2, ax=ax)
        sns.stripplot(data = df, x = 'drug', y=plot_type, order=dlabels, size=3, color='black', ax=ax, zorder=1)
    elif violinplot_overlay:
        savepath=savepath+'_violin'
        sns.violinplot(data = df, x = 'drug', y=plot_type , order=dlabels, zorder=2, linewidth=0.5, saturation=0.5, color='steelblue', ax=ax)
        sns.stripplot(data = df, x = 'drug', y=plot_type, order=dlabels, size=3, color='black', ax=ax, zorder=1)
    else:
        savepath=savepath+'_scatter'
        sns.stripplot(data = df, x = 'drug', y=plot_type, order=dlabels, size=3, color='black', ax=ax, zorder=1)
    sns.despine()
    ax.set_xlabel('Drug')
    ax.set_ylabel("$\Delta$" + ylabel + units[i])
    test = add_stat_annotation(ax, data=df, x='drug', y=plot_type, order=dlabels,
                                box_pairs=box_pairs, test='Mann-Whitney', text_format='star',
                                   line_height=0.02, text_offset=0.01, line_offset=0.01, line_offset_to_box=0.02,
                                   loc='outside', linewidth=1, verbose=0)
    
    
    print (ylabel)
    for p in box_pairs:
        print(p)
        print(stats.ttest_ind(df[df['drug'] == p[0]][plot_type], df[df['drug'] == p[1]][plot_type], equal_var=False))
        print(stats.mannwhitneyu(df[df['drug'] == p[0]][plot_type], df[df['drug'] == p[1]][plot_type]))

    fig.savefig(Path(savepath+'.png'), bbox_inches='tight')
    plt.savefig(Path(savepath+'.svg'), format='svg', bbox_inches='tight')


In [ ]:
# plot values for control by experiment
violinplot_overlay=True
df = df_list[3]
drug = 'control-combined'
datas = ['APD90_avg', 'period', 'c_APD90_Bazett', 't_rise_avg']
ylims = [(0.3, 0.7), (1.4, 2.9), (0.2, 0.5), (-0.005, 0.02)]
labels = ['APD90', 'CL', 'cAPD90', 't-depol']
units = ['/s', '/s' ,'', '/s']
order = ['before', 'after']
box_pairs = [('before', 'after')]
folders = df['folder'].unique()

for j in range(len(folders)):
    folder = folders[j]
    print(folders[j])
    df_sel = df[df['folder'] == folder]
    print(df_sel.shape[0]//2)
    for i in range(len(datas)):
        y_data = datas[i]
        print(y_data)
        ylabel = labels[i]
        y_lims = ylims[i]
        unit = units[i]
        fig, ax = plt.subplots(figsize=(1,2), dpi=600)
        savepath = drug+'/indiv/'+ylabel+'_'+folders[j][17:]
        
        if y_data == 't_rise_avg':
            df1 = df_sel
            df2 = df1[df1['timepoint']=='before']
            df2 = df2[df2['t_rise_avg'] > 0.015]
            df_sel = pd.merge(df1, df2, on=['labels', 'APD90_avg','RL_CL', 'period', 
                                'c_APD90_Bazett','t_rise_avg', 'timepoint'], how='outer', indicator=True)
            df_sel = df_sel.query("_merge != 'both'").drop('_merge', axis=1)
            df_sel = df_sel.groupby(['labels']).filter(lambda x: len(x)>1).reset_index(drop=True)
            print('dropped '+str(df2.shape[0])+' datapoints')
            print('remaining: '+str(df_sel.shape[0]//2)+ ' datapoints')
            

        if violinplot_overlay:
            savepath=savepath+'_violin'
            sns.violinplot(data = df_sel, x = 'timepoint', y = y_data , order=order, zorder=2, linewidth=0.5, saturation=0.5, color='steelblue', ax=ax)
            sns.stripplot(data = df_sel, x = 'timepoint', y=y_data, order=order, size=3, color='black', ax=ax, zorder=1)
            #plot connections
            idx0 = 4
            idx1 = 5
            locs1 = ax.get_children()[idx0].get_offsets()
            locs2 = ax.get_children()[idx1].get_offsets()
            for i in range(locs1.shape[0]):
                x = [locs1[i, 0], locs2[i, 0]]
                y = [locs1[i, 1], locs2[i, 1]]
                ax.plot(x, y, color="black", linewidth=0.5, alpha=0.1)
        else:
            savepath=savepath+'_scatter'
            sns.stripplot(data = df_sel, x = 'timepoint', y=y_data, order=order, size=3, color='black', ax=ax, zorder=1)    

            #plot connections
            idx0 = 0
            idx1 = 1
            locs1 = ax.get_children()[idx0].get_offsets()
            locs2 = ax.get_children()[idx1].get_offsets()
            for i in range(locs1.shape[0]):
                x = [locs1[i, 0], locs2[i, 0]]
                y = [locs1[i, 1], locs2[i, 1]]
                ax.plot(x, y, color="black", linewidth=0.5, alpha=0.1)

        sns.despine()
        ax.set_xlabel('Timepoint')
        ax.set_ylabel(ylabel+unit)
        ax.set_ylim(y_lims)
        
        bef = df_sel[df_sel['timepoint'] == 'before']
        aft = df_sel[df_sel['timepoint'] == 'after']
        diff = aft[y_data].to_numpy() - bef[y_data].to_numpy()
        test='t-test_paired'    
        if stats.shapiro(diff)[1] < 0.05:
            test = 'Wilcoxon'
            
        test = add_stat_annotation(ax, data=df_sel, x='timepoint', y=y_data, order=order,
                                   box_pairs=box_pairs, test=test, text_format='star',
                                   line_height=0.02, text_offset=0.01, line_offset=0.01, line_offset_to_box=0.02,
                                   loc='outside', linewidth=1, verbose=0)
        plt.suptitle('Experiment '+str(j), x=0.5, y=1.05)

        plt.savefig(Path(savepath+'.png'), format='png', bbox_inches='tight')
        plt.close()
        

In [ ]:
drug_folders = ['Quin-10uM', 'Flec-3uM', 'Lido-100uM', 'control-combined']
i=3
df = df_list[i]
drug = drug_folders[i]
bef = df[df['timepoint'] == 'before']
aft = df[df['timepoint'] == 'after']
bef['c_APD90_Bazett'].to_csv('C:/Users/pokem/Desktop/yang NEA data/single dose plot paired/bef.csv')
aft['c_APD90_Bazett'].to_csv('C:/Users/pokem/Desktop/yang NEA data/single dose plot paired/aft.csv')
stats.ttest_ind(bef['c_APD90_Bazett'],aft['c_APD90_Bazett'],equal_var=True )

In [ ]:
dfs = pd.concat(df_list)
dfs = dfs[dfs['timepoint'] == 'before']
fig, ax = plt.subplots(figsize=(3,2), dpi=600)
sns.histplot(data=dfs, x = 't_rise_avg')
sns.despine()
ax.set_xlabel('depolarization/s')
print(dfs.shape[0])
print(len(dfs['folder'].unique()))
#print(stats.skewtest(dfs['t_rise_avg']))
#print(stats.kurtosistest(dfs['t_rise_avg']))
#print(stats.shapiro(dfs['t_rise_avg']))
print(stats.bootstrap([dfs['t_rise_avg'].to_numpy()], np.mean))